In [1]:
# coding: utf-8

import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import sys
import glob
import json

IMG_SIZE = (180, 320)
drop_ratio=0.2

In [6]:
class VideoAnalyze():
    def __init__(self,video_name):
        # 特徴量抽出
        # A-KAZE検出器の生成
#         self.detector = cv2.AKAZE_create()
#         self.detector = cv2.KAZE_create()
#         self.detector = cv2.ORB_create()
        # Brute-Force Matcher生成
#         self.bf=cv2.BFMatcher(cv2.NORM_L1,crossCheck=False)
        self.cascade = cv2.CascadeClassifier('cascade.xml') #分類器の指定
        self.video_name=video_name
        # 8近傍の定義
        self.neiborhood8 = np.array([[1, 1, 1],[1, 1, 1],[1, 1, 1]],np.uint8)

    def show(self, match):
        img1=match['img1']
        img2=match['img2']
        plt.figure(figsize=(16,12))
        # 左
        plt.subplot(2,2,1)
        plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
        # 右
        plt.subplot(2,2,2)
        plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
        plt.show()
        

    def show_matching(self, match):
        matched=match['matched']
        kp1=match['kp1']
        kp2=match['kp2']
        img1=match['img1']
        img2=match['img2']
        # 対応する特徴点同士を描画
        img = cv2.drawMatchesKnn(img1, kp1, img2, kp2, matched, None, flags=2)
#     img = cv2.drawMatches(img1, kp1, img2, kp2, matched, None, flags=2)
        plt.figure(figsize=(16,12))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.show()

#     def compare_frame(self, frame, frame_path):
    def compare_frame(self, frame):
        # 特徴量の検出と特徴量ベクトルの計算
        self.top_match={}
        gray_tmp=frame.copy()
#         gray_tmp = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
        gray_tmp = cv2.resize(gray_tmp, IMG_SIZE)
        gray_tmp=gray_tmp[10:320, 0:180]
#         __, target_des = self.detector.detectAndCompute(gray_tmp, None)
        y=310
        x=180
#         target_hist = cv2.calcHist([gray_tmp], [0], None, [256], [0, 256])
        target_hist1 = cv2.calcHist([gray_tmp[0:int(y/2), 0:int(x/2)]], [0], None, [256], [0, 256])
        target_hist2 = cv2.calcHist([gray_tmp[0:int(y/2), int(x/2):x]], [0], None, [256], [0, 256])
        target_hist3 = cv2.calcHist([gray_tmp[int(y/2):y, 0:int(x/2)]], [0], None, [256], [0, 256])
        target_hist4 = cv2.calcHist([gray_tmp[int(y/2):y, int(x/2):x]], [0], None, [256], [0, 256])
        for path in glob.glob('./base_img/*.png'):
            try:
    #             matches = bf.match(target_des, d_dict[path]['des'])
    #             dist = [m.distance for m in matches]
    #             matched = [m for m in matches]
    #             matched = sorted(matches, key=lambda x:x.distance)

#                 matches = self.bf.knnMatch(target_des, self.d_dict[path]['des'], k=2)
#                 dist = []
#                 matched=[]
#                 for m, n in matches:
#                     if m.distance < drop_ratio * n.distance:
#                         matched.append([m])
#                         dist.append(m.distance)

#                 ret = sum(dist) / len(dist)
                ret=[]
#                 ret.append(cv2.compareHist(target_hist, self.d_dict[path]['hist'], cv2.HISTCMP_CHISQR_ALT))
                ret.append(cv2.compareHist(target_hist1, self.d_dict[path]['hist1'], cv2.HISTCMP_CHISQR_ALT))
                ret.append(cv2.compareHist(target_hist2, self.d_dict[path]['hist2'], cv2.HISTCMP_CHISQR_ALT))
                ret.append(cv2.compareHist(target_hist3, self.d_dict[path]['hist3'], cv2.HISTCMP_CHISQR_ALT))
                ret.append(cv2.compareHist(target_hist4, self.d_dict[path]['hist4'], cv2.HISTCMP_CHISQR_ALT))
#                 print("{0} : {1} : {2}".format(path, frame_path,sum(ret)))
                if not self.top_match or self.top_match['ret'] > sum(ret):
                    self.top_match['ret']=sum(ret)
                    self.top_match['path']=path
                    self.top_match['ts']={'img1':self.d_dict[path]['img'],'img2':gray_tmp}
#                     self.top_match['ts']={'matched':matched, 'img1':self.d_dict[path]['img'], \
#                                           'kp1':self.d_dict[path]['kp'],'img2':gray_tmp, 'kp2':__,} 
            except cv2.error:
                ret = 100000

#         print(self.top_match['path'], self.top_match['ret'])
#         self.show_matching(self.top_match['ts'])
#         self.show(self.top_match['ts'])

    def create_base_sienario(self, frame, rect, senario_dict, i):
        tmp=frame.copy()
        for (x, y, w, h) in rect:
            if y < 30 or h > 500 or w > 500:
                continue
            # タップライン
            height, width = tmp.shape[:2]
            if h < 100:
                h_buff=int((100-h)/2)                
                y = y - h_buff
                h = h + h_buff*2
            tap_line=tmp[y:y+h, 0:width]
            tap_line_name='./s_img/tap_line_{0}.jpg'.format(i)
            cv2.imwrite(tap_line_name, tap_line)
            senario_dict[i]={}
            senario_dict[i]['line']={}
            senario_dict[i]['line']['base']=self.top_match['path']
            senario_dict[i]['line']['match']=tap_line_name
            senario_dict[i]['line']['is_parent']=True
            senario_dict[i]['line']['scale']=1

            # タップ位置
            if w < 100:
                w_buff=int((100-w)/2)
                x-=w_buff
                w+=w_buff*2
            tap_img=tmp[y:y+h, x:x+w]
            tap_name='./s_img/tap_{0}.jpg'.format(i)
            cv2.imwrite(tap_name, tap_img)
            senario_dict[i]['tap']={}
            senario_dict[i]['tap']['base']=None
            senario_dict[i]['tap']['match']=tap_name
            senario_dict[i]['tap']['scale']=1
            i+=1
        return i

    def flame_diff(self,im1,im2,im3,th, tl,blur):
        im1=cv2.cvtColor(im1, cv2.COLOR_RGB2GRAY)
        im2=cv2.cvtColor(im2, cv2.COLOR_RGB2GRAY)
        im3=cv2.cvtColor(im3, cv2.COLOR_RGB2GRAY)

        d1 = cv2.absdiff(im3, im2)
        d2 = cv2.absdiff(im2, im1)
        diff = cv2.bitwise_and(d1, d2)
        
        # 差分が閾値より小さければTrue
        #mask = np.where((diff<th)&(diff>tl))
        mask = diff<th

        # 背景画像と同じサイズの配列生成
        im_mask = np.empty((im1.shape[0],im1.shape[1]),np.uint8)
        im_mask[:][:]=255
        # Trueの部分（背景）は黒塗り
        im_mask[mask]=0

        # 8近傍で膨張処理
        im_mask = cv2.dilate(im_mask,
                              self.neiborhood8,
                              iterations=5)
        # ノイズ除去
        im_mask = cv2.medianBlur(im_mask,blur)

        return  im_mask

    def find_rect(self,image):
    #  hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV_FULL)
    #  h = hsv[:, :, 0]
    #  s = hsv[:, :, 1]
    #  mask = np.zeros(h.shape, dtype=np.uint8)
    #  mask[((h < 20) | (h > 200)) & (s > 128)] = 255
        _, contours, _ = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        rects = []
        for contour in contours:
            approx = cv2.convexHull(contour)
            rect = cv2.boundingRect(approx)
            rects.append(np.array(rect))
        return rects
    
    def main(self):
        senario_dict={}
        self.d_dict={}
        for img_path in glob.glob('./base_img/*.png'):
            gray_img = cv2.imread(img_path)
#             gray_img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            gray_img = cv2.resize(gray_img, IMG_SIZE)
            gray_img=gray_img[10:320, 0:180]
#             kp, des = self.detector.detectAndCompute(gray_img, None)          
            self.d_dict[img_path]={}
#             self.d_dict[img_path]['des']=des
#             self.d_dict[img_path]['kp']=kp
            self.d_dict[img_path]['img']=gray_img
            y=310
            x=180
#             self.d_dict[img_path]['hist'] = cv2.calcHist([gray_img], [0], None, [256], [0, 256])
            self.d_dict[img_path]['hist1'] = cv2.calcHist([gray_img[0:int(y/2), 0:int(x/2)]], [0], None, [256], [0, 256])
            self.d_dict[img_path]['hist2'] = cv2.calcHist([gray_img[0:int(y/2), int(x/2):x]], [0], None, [256], [0, 256])
            self.d_dict[img_path]['hist3'] = cv2.calcHist([gray_img[int(y/2):y, 0:int(x/2)]], [0], None, [256], [0, 256])
            self.d_dict[img_path]['hist4'] = cv2.calcHist([gray_img[int(y/2):y, int(x/2):x]], [0], None, [256], [0, 256])
        i=0
        ct=0
        is_print=True
        cap = cv2.VideoCapture(self.video_name)
        _, frame1 = cap.read()
        _, frame2 = cap.read()
        ret, frame3 = cap.read()
        #print("{0}: is Open {1}".format(self.video_name, cap.isOpened()))
#         prev_detect=np.array([[-1,-1,-1,-1]])
        while ret:
#         for frame_path in glob.glob('./cat_img/*.png'):
#             ret, frame = cap.read()
#             frame = cv2.imread(frame_path)
#             if frame is None:
#                 break
            frame_fs = self.flame_diff(frame1.copy(),frame2.copy(),frame3.copy(), 5, 0,5)
            rect = self.find_rect(frame_fs.copy())
            ct+=1
            if ct>10:
                is_print=True

            if is_print and len(rect) and len(rect) <10:
                is_print=False
                ct=0
                print(len(rect), rect)
                label = cv2.connectedComponentsWithStats(frame_fs)
                # ラベルの個数nと各ラベルの重心座標cogを取得
                n = label[0] - 1
                cog = np.delete(label[3], 0, 0)
                #print(cog)
                #print(int(cog[:,0].mean()))
                #print(int(cog[:,1].mean()))
                #break
                # 重心に赤円を描く
                #for i in range(n):
                #im2 = cv2.circle(im2,(int(rect[:,0].mean()),int(rect[:,1].mean())), 40, (0,0,0), -1)
                test2 = cv2.circle(frame2.copy(),(int(cog[:,0].mean()),int(cog[:,1].mean())), 40, (0,0,0), -1)
                cv2.imwrite("./test/test{0:0>3}.jpg".format(i) ,test2)

            #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # detectMultiScale(Mat image, MatOfRect objects, double scaleFactor, int minNeighbors, int flag, Size minSize, Size maxSize)
#             detect = self.cascade.detectMultiScale(frame, 1.1, 3) #物体の検出
#             print (detect,prev_detect)
#             if type(detect) is tuple or (detect == prev_detect).all():
#                 prev_detect= np.array([[-1,-1,-1,-1]]) if type(detect) is tuple else detect
#                 continue
#             print(detect)
#             self.compare_frame(frame, frame_path)
                self.compare_frame(frame2)
                i=self.create_base_sienario(frame2, rect, senario_dict, i)

            frame1 = frame2
            frame2 = frame3
            ret, frame3 = cap.read()
#             prev_detect=detect
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        # print(senario_dict)
#         cap.release()
        return senario_dict

In [7]:
# テストベースシナリオ
if __name__ == '__main__':
    va=VideoAnalyze('./test.mp4')
    senario_dict=va.main()
    print(senario_dict)
    f = open('{0}.json'.format("base_senario"), 'w')
    json.dump(senario_dict, f)
    f.close()

8 [array([ 415, 1619,   13,   11]), array([ 725, 1165,   16,   11]), array([605, 808,  22,  11]), array([713, 738,  12,  11]), array([411, 456,  24,  13]), array([878, 279,  23,  12]), array([850, 275,  14,  11]), array([ 30, 271,  11,  11])]
9 [array([1071,    0,    9,    7]), array([1019,    0,   38,    7]), array([975,   0,  29,   9]), array([858,   0,  70,   7]), array([625,   0, 150,   7]), array([299,   0, 322,   7]), array([254,   0,  30,   7]), array([ 85,   0, 165,   7]), array([ 0,  0, 79,  8])]
7 [array([ 239, 1680,   13,   13]), array([ 632, 1547,   11,   11]), array([ 841, 1240,   11,   11]), array([301,  30,  11,  11]), array([637,  24,  12,  11]), array([621,  24,  11,  11]), array([163,  24,  17,  11])]
1 [array([815,  22,  11,  16])]
1 [array([307, 967,  16,  84])]
2 [array([787,  21,  49,  39]), array([807,  36,  13,  10])]
{0: {'line': {'base': './base_img/Screenshot_20170620-194415.png', 'match': './s_img/tap_line_0.jpg', 'is_parent': True, 'scale': 1}, 'tap': {'bas